In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import py3Dmol

from proteome import protein
from proteome.models.protein_seq_des import config
from proteome.models.protein_seq_des.modeling import ProteinSeqDesForSequenceDesign, PSD_MODEL_CONFIGS
from proteome.models.omegafold.modeling import OmegaFoldForFolding

In [ ]:
designer = ProteinSeqDesForSequenceDesign("conditional_model_0", random_seed=0)
folder = OmegaFoldForFolding()

In [ ]:
with open("5L33.pdb", mode="r") as f:
    gt_pdb = f.read()

In [ ]:
target_protein = protein.from_pdb_string(gt_pdb)

In [ ]:
sequence, energy = designer.design_sequence(
    target_protein, 
    inference_config=config.InferenceConfig(sampler_config=config.SamplerConfig(is_tim=False)),
)
print(f"Sequence: {sequence} with energy {energy:.3f}")

In [ ]:
predicted_protein, confidence = folder.fold(sequence)
result_pdb = protein.to_pdb(predicted_protein)

In [ ]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(result_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()